# Explore Sentinel-1 Archive for Uturuncu Batch Processing

In [21]:
import geopandas as gpd
import pandas as pd
import geoviews as gv
import hvplot.pandas

In [26]:
# if dinosar library not in base environment uncomment below (run just once)
!pip install --no-cache git+https://github.com/scottyhq/dinosar.git@master

  Cloning https://github.com/scottyhq/dinosar.git (to revision master) to /tmp/pip-req-build-kuvta9en
  Running command git clone -q https://github.com/scottyhq/dinosar.git /tmp/pip-req-build-kuvta9en
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dinosar: filename=dinosar-0.0.0-py3-none-any.whl size=20268 sha256=642f01858025816f17cc23def9ad651cd339709b5dbdadb24ee270da213b699b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rdos_btp/wheels/92/b0/87/750fc305aa2cac58744b55287e227c6fbf08fb234cb46ca2da
Successfully built dinosar


In [24]:
import dinosar
from importlib.metadata import version
version('dinosar')

'0.0.0'

In [8]:
gf = gpd.read_file('apmb.geojson')
snwe = dinosar.archive.asf.ogr2snwe('apmb.geojson')
snwe

[-23.68699916256815,
 -21.29081519506481,
 -68.46349117379683,
 -65.27091018760244]

In [9]:
# Put these on a map
S,N,W,E = [-25, -20, -72, -65]
tiles = gv.tile_sources.CartoEco.options(width=700, height=500).redim.range(Latitude=(S, N), Longitude=(W, E)) 
aoi = gf.hvplot(geo=True, fill_color=None, line_color='orange', title='Sentinel-1 Area of Interest', hover=False)
utur = gv.Points([(-67.18, -22.27)]).opts(color='r', marker='^', size=10)
tiles * aoi * utur

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [x,y]

In [10]:
# Plot S1 Footprints from S1 Archive (query.geojson)
def refresh_inventory(snwe):
    dinosar.archive.asf.query_asf(snwe, 'SA')
    dinosar.archive.asf.query_asf(snwe, 'SB')
    gf = dinosar.archive.asf.merge_inventories('query_SA.json', 'query_SB.json')
    dinosar.archive.asf.save_inventory(gf)

In [11]:
# Run if you want an up-to-date version of query.geojson
#refresh_inventory(snwe)

In [12]:
gf = dinosar.archive.asf.load_inventory('query.geojson')

In [13]:
gf.relativeOrbit.unique()

array([  3,  10,  76,  83, 149, 156])

In [14]:
gft = gf[['relativeOrbit','geometry']].dissolve(by='relativeOrbit').reset_index()
gft['code'] = gft.relativeOrbit.astype('category').cat.codes
gft.head()

,relativeOrbit,geometry,code
0,3,"POLYGON ((-64.90537 -24.27037, -64.90618 -24.2...",0
1,10,"POLYGON ((-66.22261 -25.02893, -66.19150 -24.9...",1
2,76,"POLYGON ((-66.50769 -25.52393, -66.50785 -25.5...",2
3,83,"POLYGON ((-65.40583 -24.03589, -65.40540 -24.0...",3
4,149,"POLYGON ((-66.14912 -24.84245, -66.14906 -24.8...",4


## Plots to summarize Sentinel-1 Inventory

In [15]:
# Want to Omit tracks 10 and 3 (too far east)
tracks = [76,83,149,156]
gf = gf[gf.relativeOrbit.isin(tracks)]
gf['code'] = gf.relativeOrbit.astype('category').cat.codes
# Extract frames from particular month to get a feel for where frames are
start = '09/01/2019'
end = '09/30/2019'
gdf = gf.query('dateStamp > @start & dateStamp < @end')
#gdf['relativeOrbit'] = gdf.relativeOrbit.astype('category')

In [16]:
footprints = gdf.hvplot.polygons(geo=True, hover_cols=['code','relativeOrbit'], cmap='Category10',  
                                 alpha=0.3, legend=True, colorbar=False, title='Sentinel-1 Tracks') #color=hv.Cycle('Category20')
tiles * footprints * aoi * utur 

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (code,relativeOrbit)
   .Polygons.II :Polygons   [Longitude,Latitude]
   .Points.I    :Points   [x,y]

In [17]:
ts = gf.loc[:, ['timeStamp','code','relativeOrbit']]
ts.hvplot.scatter(by='relativeOrbit', x='timeStamp', marker='dash', size=200, angle=90, c='code', cmap='Category10', 
                  title='Sentinel-1 Tracks',
                  colorbar=False, yaxis=None, legend=True, width=800, muted_alpha=0.1, alpha=1).opts(padding=0.1)

:NdOverlay   [relativeOrbit]
   :Scatter   [timeStamp]   (code)

## Extras

In [18]:
# Illustrate different coverage of S1A and S1B frames
gdf.query('relativeOrbit == 156').groupby(['platform','dateStamp','frameNumber'])['geometry'].count()

platform     dateStamp   frameNumber
Sentinel-1A  2019-09-08  663            1
                         668            1
             2019-09-20  663            1
                         668            1
Sentinel-1B  2019-09-02  662            1
                         667            1
                         672            1
             2019-09-14  662            1
                         667            1
                         672            1
             2019-09-26  662            1
                         667            1
                         672            1
Name: geometry, dtype: int64

In [19]:
# Print all metadata for a single frame
#with pd.option_context('display.max_rows', 100):
#    print(gdf.iloc[0])
# frameNumber                                                           1110
# platform                                                       Sentinel-1B

In [20]:
# raised bug on hvplot issue tracker: https://github.com/holoviz/hvplot/issues/381
import holoviews as hv
gf_s1a = gdf.query('relativeOrbit == 156 and platform == "Sentinel-1A"')
gf_s1b = gdf.query('relativeOrbit == 156 and platform == "Sentinel-1B"')
overlays = hv.NdOverlay({'S1A':gf_s1a.hvplot(geo=True, alpha=0.5), 
                         'S1B':gf_s1b.hvplot(geo=True, alpha=0.1), 
                        })
overlays.opts(title='Sentinel-1 Track 156') * tiles * aoi * utur 

:Overlay
   .NdOverlay.I :NdOverlay   [Element]
      :Polygons   [Longitude,Latitude]
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .Points.I    :Points   [x,y]

In [ ]:
#Note that for track 156, we don't really need to include subswath 3 since it is very far west
# Subswaths for region of interest Uturuncu+moat region:
#156: 1,2
#149: 2,3
#76: 1,2
#83: 2,3